# Density evaluation

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import os
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

# Loading dataset

In [ ]:
dataset_path='C:/Users/miche/Desktop/anemia-detection/database_sclere/'   # modify the dataset_path to your own dir
assert dataset_path!='' and dataset_path!='/path/to/dataset/', 'Please specify the dataset_path!'

# load csv file
df = pd.read_csv(dataset_path+'hbvalue.csv', sep=';')

# cut the dataset only to ind and ita dataset
df = df[(df['dataset']=='ind') | (df['dataset']=='ita')]

df.head()


# Loading photos

In [ ]:
sclera_vessels = []
sclera_masks = []
sclera_cielabs = []

# cicla sul dataset indiano
for index, row in tqdm(df.iterrows(), total=df.shape[0]):

    if row['dataset'] == 'ind':
        path_photos = dataset_path+'Dataset indiano/'
        paziente = row['paziente'] + "-"
    else:
        path_photos = dataset_path+'Italiano congiuntive/Dataset congiuntive italiano segmentato/'
        paziente = row['paziente'] + "_"

    # search folder
    matching_folders = []
    for folder_name in os.listdir(path_photos):
        if folder_name.startswith(paziente) and os.path.isdir(os.path.join(path_photos, folder_name)):
            path_single_photo = os.path.join(path_photos, folder_name)

            name_photo = folder_name.split("-")[-1]
            # load sclera
            sclera_mask = cv2.imread(path_single_photo + "/" + name_photo + '_sclera.png')
            sclera_masks.append(sclera_mask)

            # converts to cielab
            sclera_cielab = cv2.cvtColor(sclera_mask, cv2.COLOR_BGR2LAB)
            sclera_cielabs.append(sclera_cielab)

            # load mask
            sclera_vessel = cv2.imread(path_single_photo + "/" + name_photo + '_sclera_vessels_auto_deep.png', cv2.IMREAD_UNCHANGED)

            if sclera_vessel is None:
                sclera_vessel = cv2.imread(path_single_photo + "/" + name_photo + '_sclera_vessels_auto.png', cv2.IMREAD_UNCHANGED)
            
            sclera_vessels.append(sclera_vessel)
            break



# Preprocessing

In [ ]:
sclera_vessels_shape = sclera_vessels[0].shape

# converts the sclera masks to black and white
for index, mask in tqdm(enumerate(sclera_masks), total=len(sclera_masks)):
    # resize the mask to the sclera vessel shape
    mask = cv2.resize(mask, sclera_vessels_shape[:2][::-1])
    
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    
    _, mask = cv2.threshold(mask, 2, 255, cv2.THRESH_BINARY)

    sclera_masks[index] = mask

# converts the sclera vessels to black and white
for index, mask in tqdm(enumerate(sclera_vessels), total=len(sclera_vessels)):
    # transparence becomes black
    mask[mask[:, :, 3] == 0] = [0, 0, 0, 0]
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    
    _, mask = cv2.threshold(mask, 2, 255, cv2.THRESH_BINARY)
    
    sclera_vessels[index] = mask


# show the sclera masks and sclera vessels
for i in range(5):
    plt.figure(figsize=(20, 10))
    plt.subplot(1, 2, 1)
    plt.imshow(sclera_masks[i], cmap='gray')
    plt.subplot(1, 2, 2)
    plt.imshow(sclera_vessels[i], cmap='gray')
    plt.show()

In [ ]:
def increase_contrast_lab(image_lab):
    image_lab[:, :, 1] = cv2.multiply(image_lab[:, :, 1], 1.2)

    return image_lab

for index, image in tqdm(enumerate(sclera_cielabs), total=len(sclera_cielabs)):

    # Applica l'equalizzazione dell'istogramma adattivo a tutti i canali (L, a, b) dell'immagine Lab
    clahe = cv2.createCLAHE(clipLimit=1.82, tileGridSize=(8, 8))
    image_lab_eq = image.copy()
    for i in range(3):
        image_lab_eq[:, :, i] = clahe.apply(image_lab_eq[:, :, i])

    # Applica il contrasto all'immagine Lab
    image_lab_eq = increase_contrast_lab(image_lab_eq)

    sclera_cielabs[index] = image_lab_eq
    


# Density calculation

In [ ]:
# calculate the density of the color of vessels in the sclera
vessels_colors_density = []
vessels_colors_quantiles = []
vessels_colors_density_rgb = []
vessels_colors_deviations = []
for index, mask in tqdm(enumerate(sclera_vessels), total=len(sclera_vessels)):
    
    cielab = sclera_cielabs[index]

    mask =  cv2.resize(mask, cielab.shape[:2][::-1])

    # converts to rgb
    cielab_rgb = cv2.cvtColor(cielab, cv2.COLOR_LAB2BGR)
    cielab_rgb = cv2.cvtColor(cielab_rgb, cv2.COLOR_BGR2RGB)

    # moltiplica la maschera per le sclere con i colori
    cielab = cv2.bitwise_and(cielab, cielab, mask=mask)
    cielab_rgb = cv2.bitwise_and(cielab_rgb, cielab_rgb, mask=mask)
    
    # calcola la densità dei colori
    densita = cv2.mean(cielab, mask=mask)
    densita_rgb = cv2.mean(cielab_rgb, mask=mask)
    vessels_colors_density.append(densita)
    vessels_colors_density_rgb.append(densita_rgb)

    std = cv2.meanStdDev(cielab, mask=mask)[1]
    vessels_colors_deviations.append(std)

    quantile = np.empty((3, 3))
    filtered_cielab = cielab[:, :, 0]
    filtered_cielab = filtered_cielab[filtered_cielab != 0]
    # calcola i quantili
    quantile[0] = np.quantile(filtered_cielab, [0.25, 0.5, 0.75])

    filtered_cielab = cielab[:, :, 1]
    filtered_cielab = filtered_cielab[filtered_cielab != 0]
    # calcola i quantili
    quantile[1] = np.quantile(filtered_cielab, [0.25, 0.5, 0.75])

    filtered_cielab = cielab[:, :, 2]
    filtered_cielab = filtered_cielab[filtered_cielab != 0]
    # calcola i quantili
    quantile[2] = np.quantile(filtered_cielab, [0.25, 0.5, 0.75])
    vessels_colors_quantiles.append(quantile)



# calculate the density of the color of not the vessels in the sclera
vessels_colors_white_density_rgb = []
vessels_colors_white_density_cielab = []
vessels_colors_white_quantiles_cielab = []
vessels_colors_white_deviations_rgb = []
vessels_colors_white_deviations_cielab = []
for index, mask in tqdm(enumerate(sclera_vessels), total=len(sclera_vessels)):
        
        cielab = sclera_cielabs[index]
        # converts to rgb
        cielab_rgb = cv2.cvtColor(cielab, cv2.COLOR_LAB2BGR)
        cielab_rgb = cv2.cvtColor(cielab_rgb, cv2.COLOR_BGR2RGB)
        

        mask =  cv2.resize(mask, cielab_rgb.shape[:2][::-1])
        mask2 =  cv2.resize(sclera_masks[index], cielab_rgb.shape[:2][::-1])
        maschera = cv2.bitwise_not(mask)
        # metti in and la maschera con sclera_mask
        maschera = cv2.bitwise_and(maschera, maschera, mask=mask2)

        # moltiplica la maschera per le sclere con i colori
        cielab = cv2.bitwise_and(cielab, cielab, mask=maschera)
        cielab_rgb = cv2.bitwise_and(cielab_rgb, cielab_rgb, mask=maschera)

        # calcola la media dei colori
        densita_rgb = cv2.mean(cielab_rgb, mask=cv2.bitwise_not(mask))
        densita_cielab = cv2.mean(cielab, mask=cv2.bitwise_not(mask))
        vessels_colors_white_density_rgb.append(densita_rgb)
        vessels_colors_white_density_cielab.append(densita_cielab)
        
        
        quantile = np.empty((3, 3))
        filtered_cielab = cielab[:, :, 0]
        filtered_cielab = filtered_cielab[filtered_cielab != 0]
        # calcola i quantili
        quantile[0] = np.quantile(filtered_cielab, [0.25, 0.5, 0.75])

        filtered_cielab = cielab[:, :, 1]
        filtered_cielab = filtered_cielab[filtered_cielab != 0]
        # calcola i quantili
        quantile[1] = np.quantile(filtered_cielab, [0.25, 0.5, 0.75])

        filtered_cielab = cielab[:, :, 2]
        filtered_cielab = filtered_cielab[filtered_cielab != 0]
        # calcola i quantili
        quantile[2] = np.quantile(filtered_cielab, [0.25, 0.5, 0.75])
        vessels_colors_white_quantiles_cielab.append(quantile)


        std = cv2.meanStdDev(cielab_rgb, mask=mask)[1]
        vessels_colors_white_deviations_rgb.append(std)
        std = cv2.meanStdDev(cielab_rgb, mask=mask)[1]
        vessels_colors_white_deviations_cielab.append(std)

# Logistic regression

In [ ]:
# importa naive bayes
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

# prepara le features per il modello
X = []
# cicla sulle metriche (densità, quantili, deviazioni standard, ...) per ricavare le feature
for index, (
    color_density,
    color_quantiles,
    color_deviation,
    white_deviation_cielab,
    white_quantiles_cielab
) in tqdm(
    enumerate(zip(
        vessels_colors_density,
        vessels_colors_quantiles,
        vessels_colors_deviations,
        vessels_colors_white_deviations_cielab,
        vessels_colors_white_quantiles_cielab
    )),
    total=len(vessels_colors_density)
):
    # Feature light
    light_vessel = color_quantiles[0][1]                                        # mediana del canale L dei vasi sanguigni
    diff_light = white_quantiles_cielab[0][2] - color_quantiles[0][2]           # diff. tra i quantili 75 deo canale L
    diff_deviations_light = color_deviation[0] - white_deviation_cielab[0]      # diff. tra le deviaz. dei canali L

    # Feature red
    A_star = color_density[1]                                                   # A_star dei vasi sanguigni
    diff_red = color_quantiles[1][2] - white_quantiles_cielab[1][2]             # diff. tra i quantili 75 del canale A*

    # Feature blue
    diff_blue = white_quantiles_cielab[2][0] - color_quantiles[2][2]            # diff. tra il quantile 25 del canale B della parte bianca e 75 dei vasi sanguigni
    diff_deviations_blue = color_deviation[2] - white_deviation_cielab[2]       # diff. tra le deviaz. del canale B

    # aggiungi le feature all'array X
    X.append([
        A_star, diff_red, # feature red
        light_vessel, diff_light, diff_deviations_light,  # feature light
        diff_blue, diff_deviations_blue,  # feature blue
    ])


# Costruisci l'array delle y
y = []
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    
    # preleva il valore di emoglobina e lo converte in float
    hb = float(row['hb'].replace(',', '.'))

    # se hb è minore di 12.5 allora è anemico
    if hb < 12.5:
        y.append(1)
    else:
        y.append(0)

# train the model
clf = SVC(kernel='linear', C=1, random_state=0)

# validation
scores = cross_val_score(clf, X, y, cv=5, scoring='f1')
print("f1: ", scores.mean())

scores = cross_val_score(clf, X, y, cv=5, scoring='recall')
print("recall: ", scores.mean())

scores = cross_val_score(clf, X, y, cv=5, scoring='precision')
print("precision: ", scores.mean())

scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
print("accuracy: ", scores.mean())

scores = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
print("roc_auc: ", scores.mean())
